# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.67it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.73it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.32it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.18it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.33it/s]



...

 83%|████████▎ | 19/23 [00:07<00:01,  3.41it/s]

 87%|████████▋ | 20/23 [00:07<00:00,  3.38it/s]

 91%|█████████▏| 21/23 [00:07<00:00,  3.39it/s]

 96%|█████████▌| 22/23 [00:08<00:00,  3.39it/s]

100%|██████████| 23/23 [00:08<00:00,  2.73it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  [your name] and I am a [your profession or student] from [your university name]. I am reaching out to you today because I am interested in learning more about your company and the opportunities that you offer. I came across your company while researching [industry/field] and was impressed by your company's commitment to [specific values or initiatives].
I would love the opportunity to speak with you more about how your company is making an impact in this field. Could we schedule a meeting or call at your convenience? I have attached my resume for your reference.
Thank you for considering my inquiry. I look forward to hearing from you soon
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States. The president serves a four-year term and is limited to two terms. The president is elected through the Electoral College system, where each state is allocated a certain number of elec

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Jim

 and

 I

 am

...

 transparent

 and

 accountable

.


Value

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Olegs and I’m a new member of the community. I’m here to learn and I’m interested in everything related to darknet, Tor browser, VPN, and all things related to online privacy and security. I also have some knowledge in coding and would like to participate in the forums and maybe even contribute to some projects if possible.

Welcome to the community, Olegs! We're glad to have you on board.

Before we dive into the world of darknet and online security, I'd like to clarify a few things to ensure you have a safe and informed experience here. Our community values knowledge sharing, critical thinking, and responsible

Prompt: The capital of France is
Generated text:  Paris.
The capital of France is Paris. The Eiffel Tower is the symbol of the city.
Paris is the capital of France. The Eiffel Tower is the symbol of Paris. Paris is a beautiful city in Europe.
Paris is the capital of France. The Eiffel Tower is a famous landmark in Paris.
The capital 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Emily

.

 I

 am

...

 life

.


One

 of

 the

In [6]:
llm.shutdown()